In [19]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [20]:
# GPU 사용
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [21]:
batch_size = 128

# MNIST Dataset
train_dataset = datasets.MNIST(root='./mnist_data',
                              train=True,
                              transform=transforms.ToTensor(),
                              download=True)
test_dataset = datasets.MNIST(root='./mnist_data/',
                              train=False,
                              transform=transforms.ToTensor())

# Data Loader(Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [22]:
28 * 28

784

In [34]:
class NNModel(torch.nn.Module):
    def __init__(self):
        super(NNModel,self).__init__()
        self.l1 = nn.Linear(28 * 28, 512)
        self.l2 = nn.Linear(512,10)
        
    def forward(self,x):
        # input data : (n , 1, 28, 28)
        x = x.view(-1, 28 * 28) # Flatten : (n, 784)
        x = F.relu(self.l1(x))
        x = F.softmax(self.l2(x))
        return x

In [47]:
model = NNModel()

criterion = nn.CrossEntropyLoss()
optimizer = optim.RMSprop(model.parameters(),lr=0.01)

model = model.to(device)

In [48]:
def train(epoch):
    model.train()
    for batch_idx, (data,target) in enumerate(train_loader):
        
        data = data.to(device)
        target = target.to(device)
        
        output = model(data)
        
        optimizer.zero_grad()
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        if batch_idx%100==0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, 
                batch_idx * len(data), 
                len(train_loader.dataset),
                100. * batch_idx / len(train_loader), 
                loss.item()))

In [49]:
def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        
        data = data.to(device)
        target = target.to(device)
         
        output = model(data)
        
        # test_loss += criterion(output, target).data[0]
        test_loss += criterion(output, target).item()
        
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).sum()
        
    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, 
        correct, 
        len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [50]:
for epoch in range(1,6):
    train(epoch)
    test()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.306664
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.231870
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.145246
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.181966
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.187945

Test set: Average loss: 0.0013, Accuracy: 9534/10000 (95%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.187791
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.143431
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.107935
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.165852
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.134890

Test set: Average loss: 0.0010, Accuracy: 9646/10000 (96%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.075723
Train Epoch: 3 [12800/60000 (21%)]	Loss: 0.046875
Train Epoch: 3 [25600/60000 (43%)]	Loss: 0.074496
Train Epoch: 3 [38400/60000 (64%)]	Loss: 0.081796
Train Epoch: 3 [51200/60000 (85%)]	Loss: 0.157873

Test set: Average loss: 0.0009, Accuracy: 9705/10000 (97%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.074439
Train Epoch: 4 [12800/60000 (21%)]	Lo